<a href="https://colab.research.google.com/github/Zebfred/DS-Unit-2-Regression-Classification/blob/master/w5d4_assignment2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions 

<pre>

Assignment
Do train/validate/test split with the Tanzania Waterpumps data.

 Do one-hot encoding. (Remember it may not work with high cardinality categoricals.)
 
 Use scikit-learn for logistic regression.
 
 Get your validation accuracy score.
 
 
 
 Get and plot your coefficients.
 
 
 Submit your predictions to our Kaggle competition. 
 
</pre>

#Answer

### set up

In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module4')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd

train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

In [0]:
import pandas_profiling


In [0]:
#train_features.profile_report()

###Start: Base model and accuracy score

In [0]:
train_features.head()

In [0]:
y_train = train_labels['status_group']
y_train.value_counts(normalize=True)

In [0]:
majority_class = y_train.mode()[0]
y_pred = [majority_class]* len(y_train)
print(len(y_pred))

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.543080808080808

In [0]:
from sklearn.model_selection import train_test_split

X= train_features
y= train_labels['status_group']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size=0.80, test_size=.20
    , random_state=42
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [0]:
#stratified sampling proportions in train & ShPE
y_train.value_counts(normalize=True)

In [0]:
y_val.value_counts(normalize=True)

In [0]:
X_train_numeric = X.select_dtypes('number')
X_val_numberic = X_val.select_dtypes('number')

In [0]:
#null features
X_train_numeric.isnull().sum()


In [0]:
import sklearn; sklearn.__version__

###Logistical regression

In [0]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=5000, n_jobs=-1)
model.fit(X_train_numeric, y)

In [0]:
X_val_numberic.head()

In [0]:
#evaluate

y_pred = model.predict(X_val_numberic)
accuracy_score(y_val, y_pred)


In [0]:
model.score(X_val_numberic, y_val)

In [0]:
y_pred

In [0]:
pd.Series(y_pred).value_counts(normalize=True)

###One hot Encoding

In [0]:
#one hot encoding
X_train.describe(exclude='number').T.sort_values(by='unique')

In [0]:
#explore quanity feature
X_train['quanity'].value_counts(drop=False)

#combine x and y for data exploring
train = X_train.copy(
)
train['status_group'] = y_train

#groupby

train.groupby('quantity')['status_group'].value_counts(normalize=True)

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sample_submission
#error: sns.catplot(x='quantity', y='status_group', data=train, kind='bar')

train['functional'] = (train['status_group'] =='functional').astype(int)

sns.catplot(x='quantity'), y='functional', data=train, kind='bar');

###coefficents

In [0]:
functional_coefficients = pd.Series(
    model.coef_[0],
    X_train_encoded.columns
)

plt.figure(figsize=(10, 10))
functional_coefficients.sort_values().plot.barh();

In [0]:
X_testsubset = test_features[features]
X_test_encoded = encoder.transform(X_test_subset)
X_test_scaled = scaler.transform(X_test_encoded)
assert all(X_test_\.columns == X_train_encoded.columns)

###Kaggle submissions

In [0]:
y_pred = model.predict(X_test_scaled)
submission = sample_submission.copy()
submission['status_goup'] = y_pred
submission.to_csv('submission-01.csv', index=False)

In [0]:
!head submission-01.csv

In [0]:
if in_colab:
    from google.colab import files
    # Just try again if you get this error:
    # TypeError: Failed to fetch
    # https://github.com/googlecolab/colabtools/issues/337
    files.download('submission-01.csv')